In [1]:
import numpy as np
import random, math
from scipy.optimize import minimize
from scipy.spatial import distance
import matplotlib.pyplot as plt

In [2]:
def linearKernel(x, y):
    return np.dot(x, y)

In [3]:
x = np.array([1,2,3])
y = np.array([1,3,4])
print(linearKernel(x, y))

19


In [4]:
def polynomialKernel(x, y, p):
    return np.power((1 + np.dot(x, y)), p)

In [5]:
print(polynomialKernel(x, y, 2))

400


In [6]:
def rbfKernel(x, y, sigma):
    return np.exp(-(distance.euclidean(x, y))/(2*np.power(sigma, 2)))

In [7]:
print(rbfKernel(x, y, 100))

0.9999292918218224


In [8]:
def computeP (t, x):
    p = np.zeros((len(t), len(t)))
    for i in range(len(t)):
        for j in range(len(t)):
            p[i][j] = t[i] * t[j] * linearKernel(x[i], x[j])
    return p

In [9]:
x = np.array([1,2])
y = np.array([1,3])
t = np.array([1,-1])
p = computeP(t, x)
print(p)

[[ 1. -2.]
 [-2.  4.]]


In [10]:
def objective(alpha):
    alpha_matrix = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            alpha_matrix[i][j] = alpha[i] * alpha[j]
    alpha_times_p = np.matmul(alpha_matrix, p_ij)
    sum_alpha_times_p = 0 
    for i in range(N):
        sum_alpha_times_p += np.sum(alpha_times_p[i])
    return 0.5 * sum_alpha_times_p - np.sum(alpha)

In [11]:
def zerofun(alpha): 
    return np.dot(alpha, targets)

In [13]:
np.random.seed(100)

classA = np.concatenate(
    (np.random.randn(10, 2) * 0.2 + [1.5, 0.5],
    np.random.randn(10, 2) * 0.2 + [-1.5, 0.5]))
classB = np.random.randn(20, 2) * 0.2 + [0.0, -0.5]

inputs = np.concatenate((classA, classB))
targets = np.concatenate(
    (np.ones(classA.shape[0]),
    - np.ones(classB.shape[0])))

N = inputs.shape[0]

permute = list(range(N))
random.shuffle(permute)
inputs = inputs[permute, :]
targets = targets[permute]

p_ij = computeP(targets, inputs)

In [15]:
C = None 
B = [(0, C) for b in range(N)]
ret = minimize(objective, np.zeros(N), bounds = B, constraints = {'type': 'eq', 'fun':zerofun})
print(ret['x'])

[1.33857265e-14 1.19460088e-14 1.00118758e-14 7.17187160e-16
 1.46233396e-01 0.00000000e+00 1.18321645e-14 2.16871753e-14
 9.58602917e-15 1.03614435e-14 6.99912726e-15 2.86397486e-15
 2.09338522e-14 1.41791189e-14 3.35097916e-14 0.00000000e+00
 1.60170170e-14 8.88110309e-15 0.00000000e+00 7.24821221e-15
 8.80728589e-15 1.74460109e-14 0.00000000e+00 1.85788447e-15
 6.96924464e-15 1.49202441e-14 1.16605076e-14 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.40011706e-14 1.11147603e-14
 8.41326148e-15 8.71640126e-15 1.20346002e-16 8.15119105e-15
 1.46233396e-01 1.22205166e-14 4.05755389e-14 0.00000000e+00]


In [ ]:
ret = minimize(objective1,np.zeros(40),bounds=B,constraints={'type':'eq','fun':z_const})